# B-splines FEM solver for Poisson equation (2D)


In [1]:
# needed imports
from numpy import zeros, ones, linspace, zeros_like, asarray
from matplotlib.pyplot import plot, show
%matplotlib inline

In [2]:
# ... assembling the stiffness matrix using stencil forms
def assemble_stiffness(nelements, degree, spans, basis, weights, points, matrix):

    # ... sizes
    ne1,ne2              = nelements
    p1,p2                = degree
    spans_1, spans_2     = spans
    basis_1, basis_2     = basis
    weights_1, weights_2 = weights
    points_1, points_2   = points
    
    k1 = weights_1.shape[1]
    k2 = weights_2.shape[1]
    # ...

    # ... build matrices
    for ie1 in range(0, ne1):
        i_span_1 = spans_1[ie1]        
        for ie2 in range(0, ne2):
            i_span_2 = spans_2[ie2]        
            # evaluation dependant uniquement de l'element
            
            for il_1 in range(0, p1+1):
                for il_2 in range(0, p2+1):                
                    for jl_1 in range(0, p1+1):
                        for jl_2 in range(0, p2+1):
                            i1 = i_span_1 - p1 + il_1
                            j1 = i_span_1 - p1 + jl_1

                            i2 = i_span_2 - p2 + il_2
                            j2 = i_span_2 - p2 + jl_2

                            v = 0.0
                            for g1 in range(0, k1):
                                for g2 in range(0, k2):
                                    bi_0 = basis_1[ie1, il_1, 0, g1] * basis_2[ie2, il_2, 0, g2]
                                    bi_x = basis_1[ie1, il_1, 1, g1] * basis_2[ie2, il_2, 0, g2]
                                    bi_y = basis_1[ie1, il_1, 0, g1] * basis_2[ie2, il_2, 1, g2]

                                    bj_0 = basis_1[ie1, jl_1, 0, g1] * basis_2[ie2, jl_2, 0, g2]
                                    bj_x = basis_1[ie1, jl_1, 1, g1] * basis_2[ie2, jl_2, 0, g2]
                                    bj_y = basis_1[ie1, jl_1, 0, g1] * basis_2[ie2, jl_2, 1, g2]

                                    wvol = weights_1[ie1, g1] * weights_2[ie2, g2]

                                    v += (bi_x * bj_x + bi_y * bj_y) * wvol

                            matrix[i1, j1, i2, j2]  += v
    # ...

    return matrix    
# ...

In [11]:
# ... Assembly procedure for the rhs
def assemble_rhs(f, nelements, degree, spans, basis, weights, points, rhs):

    # ... sizes
    ne1,ne2              = nelements
    p1,p2                = degree
    spans_1, spans_2     = spans
    basis_1, basis_2     = basis
    weights_1, weights_2 = weights
    points_1, points_2   = points
    
    k1 = weights_1.shape[1]
    k2 = weights_2.shape[1]
    # ...
    
    arr_f = zeros((k1,k2))

    # ... build rhs
    for ie1 in range(0, ne1):
        i_span_1 = spans_1[ie1]        
        for ie2 in range(0, ne2):
            i_span_2 = spans_2[ie2]        
            # evaluation dependant uniquement de l'element
            for g1 in range(0, k1):
                for g2 in range(0, k2):
                    x1    = points_1[ie1, g1]
                    x2    = points_2[ie2, g2]
                    
                    arr_f[g1,g2] = f(x1,x2)                    
            
            for il_1 in range(0, p1+1):
                for il_2 in range(0, p2+1):   
                    i1 = i_span_1 - p1 + il_1
                    i2 = i_span_2 - p2 + il_2                    

                    v = 0.0
                    for g1 in range(0, k1):
                        for g2 in range(0, k2):
                            bi_0 = basis_1[ie1, il_1, 0, g1] * basis_2[ie2, il_2, 0, g2]
                            bi_x = basis_1[ie1, il_1, 1, g1] * basis_2[ie2, il_2, 0, g2]
                            bi_y = basis_1[ie1, il_1, 0, g1] * basis_2[ie2, il_2, 1, g2]
                            
                            x1    = points_1[ie1, g1]
                            x2    = points_2[ie2, g2]
                            wvol  = weights_1[ie1, g1]*weights_2[ie2, g2]

                            v += bi_0 * arr_f[g1,g2] * wvol

                    rhs[i1,i2] += v
    # ...

    # ...
    return rhs
    # ...
# ...

In [4]:
from spaces import SplineSpace

In [5]:
V = SplineSpace(degree=3, nelements=32)

In [6]:
stiffness = zeros((V.nbasis, V.nbasis, V.nbasis, V.nbasis))
stiffness = assemble_stiffness((V.nelements, V.nelements), 
                               (V.degree,    V.degree), 
                               (V.spans,     V.spans), 
                               (V.basis,     V.basis), 
                               (V.weights,   V.weights), 
                               (V.points,    V.points), 
                               matrix=stiffness)

In [7]:
f = lambda x,y: 2.    

In [12]:
rhs = zeros((V.nbasis, V.nbasis))
rhs = assemble_rhs(f, 
                   (V.nelements, V.nelements), 
                   (V.degree,    V.degree), 
                   (V.spans,     V.spans), 
                   (V.basis,     V.basis), 
                   (V.weights,   V.weights), 
                   (V.points,    V.points), 
                   rhs=rhs)

In [13]:
# apply homogeneous dirichlet boundary conditions
rhs = rhs[1:-1,1:-1]
stiffness = stiffness[1:-1, 1:-1, 1:-1, 1:-1]

### Linear solver

Since we used a dense storage as a Tensor (ndarray of rank 4), we therefor can not use available linear solvers as they are. We have two options

* Convert the tensor to a sparse matrix (will be covered later)
* Define the action Matrix-Vector and give it to an iterative solver

We shall consider the second option in what follows.

Since each degree of freedom (B-Spline function) is only related to (2p+1) other dofs (because of the compact support of B-Splines), therefor we know, that our dense matrices are in fact sparse. Moreover, we know the exact shape (connectivity) of the matrix. We shall exploit this property in the sequel.

The action Matrix-Vector is then defined as

In [ ]:
def mv(M, v, degrees):
    n1,n2 = v.shape
    p1,p2 = degrees
    for i1 in range(n1):
        for i2 in range(n2):